## Perbaiki Relasi Data

Masalahnya ada pada data respon yang tidak memiliki relasi

### Petunjuk

Hanya ada pada urutan ID Auto Incement
dan tanggal (DATE)

### Solusi

dengan mengandalkan urutan id dan tanggal diharapkan dapat menentukan hubungan antara data

### Step 1
Memastikan ```Item1.id >= Item2.id && Item1.tanggal == Item2.tanggal```

In [8]:
import pandas as pd
import numpy as np
# load respondents
respondents = pd.read_csv("assets/respondents.csv")
respondents

,respondent_id,date,gender,education,job,type,category_id
0,60,2023-02-28,1,SD,NaN,Rawat Jalan,154
1,61,2023-02-28,1,SMP,NaN,Rawat Jalan,154
2,62,2023-02-28,1,SMA,NaN,Rawat Jalan,154
3,63,2023-02-28,1,SMP,NaN,Rawat Jalan,154
4,64,2023-03-01,1,SMP,NaN,Rawat Jalan,154
...,...,...,...,...,...,...,...
209,269,2024-03-01,1,D3,NaN,Rawat Jalan,154
210,270,2024-09-08,1,S3,NaN,Rawat Inap,154
211,271,2024-09-09,1,SD,NaN,Rawat Jalan,154
212,272,2024-09-12,1,SMP,NaN,Rawat Jalan,154


In [9]:
first_answers = pd.read_csv("assets/jawaban_user_1.csv")
first_answers

,id,answer,respondent_id,date,status
0,1,2,60,2023-02-28,1
1,2,1,61,2023-02-28,1
2,3,1,63,2023-02-28,1
3,4,1,64,2023-03-01,1
4,57,3,65,2023-03-25,0
...,...,...,...,...,...
193,248,1,266,2024-02-24,0
194,249,3,267,2024-03-01,0
195,250,2,270,2024-09-08,1
196,251,3,272,2024-09-12,1


In [26]:
# merge respondent with first answers
respondents_with_first_answers = pd.merge(left=respondents, right=first_answers, on='respondent_id', how='inner')
respondents_with_first_answers
respondents_with_first_answers.to_csv("output/respondents_with_first_answers.csv")

# check if there is duplicate on respondent_id column
is_respondents_with_first_answers_has_duplicate = not respondents_with_first_answers['respondent_id'].is_unique
is_respondents_with_first_answers_has_duplicate

True

### Clean Duplicate Rows

In [169]:

if is_respondents_with_first_answers_has_duplicate:
	cols = ["respondent_id"]
	respondents_with_first_answers_not_duplicate = []
	scanned_id_dict = dict()
	for index, row in respondents_with_first_answers.iterrows():
		if row['respondent_id'] not in scanned_id_dict:
			scanned_id_dict[row['respondent_id']] = 1
			respondents_with_first_answers_not_duplicate.append(row.to_list())
	respondents_with_first_answers_not_duplicate = pd.DataFrame(respondents_with_first_answers_not_duplicate, columns=respondents_with_first_answers.columns)
else:
	respondents_with_first_answers_not_duplicate = respondents_with_first_answers
respondents_with_first_answers_not_duplicate.reset_index(drop=True)
respondents_with_first_answers_not_duplicate.to_csv("output/respondents_with_first_answers_not_duplicate.csv")
first_answers = respondents_with_first_answers_not_duplicate

In [170]:
second_answers = pd.read_csv("assets/jawaban_user_2.csv")
second_answers

,id_2,answer_2,date_2,status_2
0,1,4.0,2023-02-28,1
1,2,1.0,2023-02-28,1
2,3,2.0,2023-02-28,1
3,4,2.0,2023-03-01,1
4,60,3.0,2023-03-25,0
...,...,...,...,...
179,235,1.0,2024-02-24,0
180,236,2.0,2024-03-01,0
181,237,2.0,2024-09-08,0
182,238,3.0,2024-09-12,0


## Merge second answers with first answers

In [211]:
from datetime import datetime

def merge_two_df(first: pd.DataFrame, second: pd.DataFrame, firstId: str, secondId: str, firstDate: str, secondDate: str):
	relationship = dict() 
	relationship_index = dict() 
	relationship_doesnt_belongs_to = list() 
	relationship_doesnt_belongs_to_x = list() 
	relationship_all = dict() 

	for x, rowx in first.iterrows():
		for y in range(second.shape[0]):
			rowy = second.iloc[y]
			try:
				if(rowx[firstId] == 0 or rowy[secondId] == 0):
					relationship_all[rowx[firstId]] = rowy[secondId]
				elif(int(rowx[firstId]) >= int(rowy[secondId]) and rowx[firstDate] == rowy[secondDate]):
					# print(rowx[firstId], rowy['date_2'], rowx['respondent_id'])
					if(rowx[firstId] not in relationship and rowy[secondId] not in relationship.values()):
						relationship[rowx[firstId]] = rowy[secondId]
						relationship_index[x] = y
					relationship_all[rowx[firstId]] = rowy[secondId]
				elif rowx[firstDate] == rowy[secondDate]:
					if(rowx[firstId] not in relationship and rowy[secondId] not in relationship.values()):
						relationship[rowx[firstId]] = rowy[secondId]
						relationship_index[x] = y
					relationship_all[rowx[firstId]] = rowy[secondId]
			except:
				print("cannot cast => ", rowx[firstId], rowy[secondId])

	for y in range(second.shape[0]):
		rowy = second.iloc[y]
		if(rowy[secondId] not in relationship.values()):
			relationship_doesnt_belongs_to.append(rowy[secondId])

	for x, rowx in first.iterrows():
		if(rowx[firstId] not in relationship):
			relationship_doesnt_belongs_to_x.append(rowx[firstId])
	
	return {
		'relationship': relationship,
		'relationship_index': relationship_index,
		'relationship_doesnt_belongs_to': relationship_doesnt_belongs_to,
		'relationship_doesnt_belongs_to_x': relationship_doesnt_belongs_to_x,
		'relationship_all': relationship_all,
	}



In [212]:
# merge one and answer two

def get_relation(idx: int, l: dict):
	relationship_id = None
	for i, item in enumerate(l.keys()):
		if(item == idx):
			relationship_id = l[item]
	return relationship_id

def get_relation_object(idx: int, l: pd.DataFrame, key: str):
	relation = None
	for i, item in l.iterrows():
		if(item[key] == idx):
			relation = item
	return relation

def get_result_merge(first: pd.DataFrame, result_df: dict, firstId: str, secondId: str, second: pd.DataFrame):
	first_answers_merged = list()
	for x, rowx in first.iterrows():
		idx = get_relation(rowx[firstId], result_df['relationship'])
		if(idx is not None):
			relation = get_relation_object(idx, second, secondId)
			if relation is not None:
				first_answers_merged.append(rowx.to_list() + relation.to_list())
			else:
				print("I shouldn't show up")
		else:
			first_answers_merged.append(rowx.to_list() + [0] * second.shape[1])
	for y, rowy in enumerate(result_df['relationship_doesnt_belongs_to']):
		relation = get_relation_object(rowy, second, secondId)
		if relation is not None:
			first_answers_merged.append([0] * first.shape[1] + relation.to_list())
		else:
			print("I shouldn't show up")

	merged_answers = pd.DataFrame(first_answers_merged, columns=first.columns.to_list() + second.columns.to_list())
	return merged_answers	

		


In [225]:

merge_result_df = merge_two_df(first=first_answers, second=second_answers, firstId="id", secondId="id_2", secondDate="date_2", firstDate="date_y")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_1_2.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_1_2_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=first_answers, result_df=merge_result_df, firstId="id", secondId="id_2", second=second_answers)
merged_answers.to_csv("output/merged_answers_2.csv")
merged_answers

[221, 231]


,respondent_id,date_x,gender,education,job,type,category_id,id,answer,date_y,status,id_2,answer_2,date_2,status_2
0,60,2023-02-28,1,SD,NaN,Rawat Jalan,154,1,2,2023-02-28,1,1,4.0,2023-02-28,1
1,61,2023-02-28,1,SMP,NaN,Rawat Jalan,154,2,1,2023-02-28,1,2,1.0,2023-02-28,1
2,63,2023-02-28,1,SMP,NaN,Rawat Jalan,154,3,1,2023-02-28,1,3,2.0,2023-02-28,1
3,64,2023-03-01,1,SMP,NaN,Rawat Jalan,154,4,1,2023-03-01,1,4,2.0,2023-03-01,1
4,65,2023-03-25,0,S1,NaN,Rawat Inap,154,57,3,2023-03-25,0,60,3.0,2023-03-25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,270,2024-09-08,1,S3,NaN,Rawat Inap,154,250,2,2024-09-08,1,237,2.0,2024-09-08,0
194,272,2024-09-12,1,SMP,NaN,Rawat Jalan,154,251,3,2024-09-12,1,238,3.0,2024-09-12,0
195,273,2024-09-14,1,SD,NaN,Rawat Jalan,154,252,2,2024-09-14,1,239,3.0,2024-09-14,0
196,0,0,0,0,0.0,0,0,0,0,0,0,221,3.0,2023-12-04,0


In [226]:
third_answers = pd.read_csv("assets/jawaban_user_3.csv")
third_answers
merge_result_df = merge_two_df(first=merged_answers, second=third_answers, firstId="id", secondId="id_3", secondDate="date_3", firstDate="date_2")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_2_3.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_2_3_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_3", second=third_answers)
merged_answers.to_csv("output/merged_answers_3.csv")

[1]


In [227]:
fourth_answers = pd.read_csv("assets/jawaban_user_4.csv")
fourth_answers
merge_result_df = merge_two_df(first=merged_answers, second=fourth_answers, firstId="id", secondId="id_4", secondDate="date_4", firstDate="date_x")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_3_4.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_3_4_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_4", second=fourth_answers)
merged_answers.to_csv("output/merged_answers_4.csv")

[]


In [228]:
fifth_answers = pd.read_csv("assets/jawaban_user_5.csv")
fifth_answers
merge_result_df = merge_two_df(first=merged_answers, second=fifth_answers, firstId="id", secondId="id_5", secondDate="date_5", firstDate="date_x")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_4_5.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_4_5_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_5", second=fifth_answers)
merged_answers.to_csv("output/merged_answers_5.csv")

[214]


In [229]:
sixth_answers = pd.read_csv("assets/jawaban_user_6.csv")
sixth_answers
merge_result_df = merge_two_df(first=merged_answers, second=sixth_answers, firstId="id", secondId="id_6", secondDate="date_6", firstDate="date_x")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_5_6.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_5_6_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_6", second=sixth_answers)
merged_answers.to_csv("output/merged_answers_6.csv")

[50]


In [231]:
seventh_answers = pd.read_csv("assets/jawaban_user_7.csv")
seventh_answers
merge_result_df = merge_two_df(first=merged_answers, second=seventh_answers, firstId="id", secondId="id_7", secondDate="date_7", firstDate="date_x")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_6_7.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_6_7_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_7", second=seventh_answers)
merged_answers.to_csv("output/merged_answers_7.csv")

[49, 211]


In [233]:
eighth_answers = pd.read_csv("assets/jawaban_user_8.csv")
eighth_answers
merge_result_df = merge_two_df(first=merged_answers, second=eighth_answers, firstId="id", secondId="id_8", secondDate="date_8", firstDate="date_x")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_7_8.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_7_8_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_8", second=eighth_answers)
merged_answers.to_csv("output/merged_answers_8.csv")

[49, 61, 212, 213]


In [ ]:
ninth_answers = pd.read_csv("assets/jawaban_user_9.csv")
ninth_answers
merge_result_df = merge_two_df(first=merged_answers, second=ninth_answers, firstId="id", secondId="id_9", secondDate="date_9", firstDate="date_x")

# merge_result_df
relationship_all_pd = pd.DataFrame.from_dict(merge_result_df['relationship_all'], orient='index', columns=['answer_2_id'])
relationship_all_pd = relationship_all_pd.reset_index(names="answer_1_id")
relationship_all_pd.to_csv("output/relationship_8_9.csv")

relationship_pd = pd.DataFrame.from_dict(merge_result_df['relationship'], orient='index', columns=['answer_2_id'])
relationship_pd = relationship_pd.reset_index(names="answer_1_id")
relationship_pd.to_csv("output/relationship_8_9_clean.csv")

print(merge_result_df['relationship_doesnt_belongs_to'])
merged_answers = get_result_merge(first=merged_answers, result_df=merge_result_df, firstId="id", secondId="id_9", second=ninth_answers)
merged_answers.to_csv("output/merged_answers_9.csv")